# ***YOLOv12 + MobileNetV3-small + BiFPN***

In [1]:
# Install required packages (for Google Colab)
!pip install -q ultralytics roboflow

import os
from pathlib import Path
import yaml
import torch
import torch.nn as nn
import inspect
import re
from roboflow import Roboflow
from ultralytics import YOLO
from ultralytics.nn import tasks

# Custom BiFPN Concat module
class BiFPN_Concat2(nn.Module):
    def __init__(self, dimension=1):
        super(BiFPN_Concat2, self).__init__()
        self.d = dimension
        self.w = nn.Parameter(torch.ones(2, dtype=torch.float32), requires_grad=True)
        self.epsilon = 0.0001

    def forward(self, x):
        w = self.w
        weight = w / (torch.sum(w, dim=0) + self.epsilon)
        x = [weight[0] * x[0], weight[1] * x[1]]
        return torch.cat(x, self.d)

# Register custom module
tasks.BiFPN_Concat2 = BiFPN_Concat2

# Patch parse_model to handle BiFPN_Concat2 like Concat (with indentation preservation)
source = inspect.getsource(tasks.parse_model)
pattern = r'(\s*)else:\s+c2\s*=\s*ch\s*\[f\]'
replacement = r'\1elif m is BiFPN_Concat2:\n\1 c2 = sum([ch[x] for x in f])\n\1else:\n\1 c2 = ch[f]'
modified_source = re.sub(pattern, replacement, source)
exec(modified_source, tasks.__dict__)
print("✅ Custom BiFPN_Concat2 registered and parse_model patched with fixed indentation")

# Konfigurasi Roboflow
API_KEY = "QOd5ldAdjiaehHn5m6WC"
WORKSPACE = "dentalogic8"
PROJECT_ID = "dental-caries-7kttb"
VERSION_NUM = 9

# Download Dataset
rf = Roboflow(api_key=API_KEY)
project = rf.workspace(WORKSPACE).project(PROJECT_ID)
dataset = project.version(VERSION_NUM).download("yolov12")

DATASET_DIR = Path(f"/content/dental-caries-{VERSION_NUM}")
DATA_YAML = DATASET_DIR / "data.yaml"

# Verifikasi download
if DATA_YAML.exists():
    print(f"✅ Dataset berhasil diunduh: {DATA_YAML}")
    print(f"Struktur dataset: {os.listdir(DATASET_DIR)}")
else:
    raise FileNotFoundError("Dataset gagal diunduh. Periksa koneksi dan kredensial Roboflow.")

# YOLO12n YAML configuration with MobileNetV3-Small backbone and BiFPN neck
yolo12n_yaml_content = """
nc: 7  # number of classes
scales:  # model compound scaling constants
  n: [0.50, 0.25, 1024]
  s: [0.50, 0.50, 1024]
  m: [0.50, 1.00, 512]
  l: [1.00, 1.00, 512]
  x: [1.00, 1.50, 512]

# MobileNetV3-Small backbone via TorchVision (adapted to match YOLOv12 outputs)
backbone:
  # [from, repeats, module, args]
  - [-1, 1, TorchVision, [0, mobilenet_v3_small, DEFAULT, True, 2, True]]  # 0: features
  - [0, 1, Index, [24, 3]]  # 1: P3 raw (24ch, /8)
  - [1, 1, Conv, [256, 1, 1]]  # 2: P3 projected (256ch, /8)
  - [0, 1, Index, [48, 8]]  # 3: P4 raw (48ch, /16)
  - [3, 1, Conv, [512, 1, 1]]  # 4: P4 projected (512ch, /16)
  - [0, 1, Index, [96, 12]]  # 5: P5 raw (96ch, /32)
  - [5, 1, Conv, [1024, 1, 1]]  # 6: P5 projected (1024ch, /32)

# BiFPN neck (modified from YOLO12n head with weighted fusion)
head:
  - [6, 1, nn.Upsample, [None, 2, 'nearest']]  # 7: up P5
  - [[-1, 4], 1, BiFPN_Concat2, [1]]  # 8: weighted cat up P5 + P4 projected
  - [8, 1, Conv, [512, 1, 1]]  # 9: project to 512ch
  - [9, 1, nn.Upsample, [None, 2, 'nearest']]  # 10: up projected P4
  - [[-1, 2], 1, BiFPN_Concat2, [1]]  # 11: weighted cat up P4 + P3 projected
  - [11, 1, Conv, [256, 1, 1]]  # 12: project to 256ch (P3 refined)
  - [12, 1, Conv, [256, 3, 2]]  # 13: down P3 to P4
  - [[-1, 9], 1, BiFPN_Concat2, [1]]  # 14: weighted cat down P3 + P4 top
  - [14, 1, Conv, [512, 1, 1]]  # 15: project to 512ch (P4 refined)
  - [15, 1, Conv, [512, 3, 2]]  # 16: down P4 to P5
  - [[-1, 6], 1, BiFPN_Concat2, [1]]  # 17: weighted cat down P4 + P5 projected
  - [17, 1, Conv, [1024, 1, 1]]  # 18: project to 1024ch (P5 refined)
  - [[12, 15, 18], 1, Detect, [nc]]  # Detect(P3, P4, P5)
"""

# Save YOLO12n YAML to file
yaml_path = Path("/content/yolo12n_mobilenetv3-small_bifpn.yaml")
with open(yaml_path, "w") as f:
    yaml.dump(yaml.safe_load(yolo12n_yaml_content), f)
print(f"✅ YAML model saved: {yaml_path}")

# Inisialisasi model
model = YOLO(str(yaml_path))

def train_model(model, data_yaml, project_name="train_icdas_mnetv3-small_bifpn", epochs=50):
    results = model.train(
        data=str(data_yaml),
        imgsz=640,
        epochs=epochs,
        batch=16,
        device=0,
        deterministic=True,
        project="runs/pure",
        name=project_name,
        pretrained=False
    )
    return results

train_results = train_model(model, DATA_YAML)
print("✅ Training selesai! Model terbaik: runs/pure/train_icdas_mnetv3-small_bifpn/weights/best.pt")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 116.1 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Custom BiFPN_Concat2 registered and parse_model patched with fixed indentation
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to dental-caries-9 in yolov12:: 100%|██████████| 13112/13112 [00:02<00:00, 6380.73it/s]


✅ Dataset berhasil diunduh: /content/dental-caries-9/data.yaml
Struktur dataset: ['train', 'valid', 'README.dataset.txt', 'data.yaml', 'README.roboflow.txt', 'test']
✅ YAML model saved: /content/yolo12n_mobilenetv3-small_bifpn.yaml
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth


100%|██████████| 9.83M/9.83M [00:00<00:00, 135MB/s]


Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dental-caries-9/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/yolo12n_mobilenetv3-small_bifpn.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train_icdas_mnetv3-small_bifpn, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overl

KeyboardInterrupt: 